In [14]:
from Bio import SeqIO
import pandas as pd

with open('./genome_raw/Aurelia.V.1.1.transcripts.fasta') as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        seqs.append(str(seq_record.seq))

In [15]:
df = pd.DataFrame([identifiers,seqs,lengths]).T
df.columns = ["id","seq","length"]

In [17]:
df.head()

,id,seq,length
0,Seg1.1,ATGTGGGACGAAGTTAGACGATGCGAAATTTCGGAGGCGATTACAA...,363
1,Seg1.2,TTGGTCCTAACAGTTGCAACACCTCCGTTAAAAACAGACCCATTCT...,196
2,Seg1.3,GATGAATGATCTCCAAAACTCCTTCCTAACATCTCACCAAATTGCA...,313
3,Seg1.4,AGGCGTTCTGGATCATTTAATTCCATGGATACATCCTCCTCTGTAT...,2880
4,Seg1.5,AGGAGAGAAGACAACCTGAGGCAGTTTTATCGTTATTGTAAGCTCA...,1608


In [29]:
diff_segs = pd.read_csv("./diff_seg_name.csv",header=0)

In [33]:
df_filt = pd.merge(df,diff_segs,left_on="id",right_on="x")

In [36]:
df_filt.loc[:,["id","seq"]]

,id,seq
0,Seg1.9,TGGTATGAGATTGATGCTTTTGGACTACCAAATCAACTGCGGCAAA...
1,Seg10.1,TTTTTGGGACACCCTGTATTGTTATGTTACAACATCTGCTGCAGTA...
2,Seg1000.3,GCCTCACTCCTTTTGAATCGACATGATTTTCCTTGTAAATAAGACT...
3,Seg1000.6_Seg1000.7,AGTTTCTTATAAAAAAAATGAGTGTTTATGAAGTTTTACAAAAATC...
4,Seg1000.18,AGTTGAACGAAAGAAAAGTGCACAAAGAAAAATGTCTGGACGTGGA...
...,...,...
1925,Seg99.3,ATGTACCCCCAAGAAGAAGCAAACATTGTTATGTTAGGTGTCAAAG...
1926,Seg992.7,GGACAACGCCCGCACCATTCACAACAACGGCTGCAGGTCCACAACC...
1927,Seg994.5,CAGTCTAGTAATATAGGGCGTGTGCAATCATTCGGAATTAACTACG...
1928,Seg997.8,CCGCACCTAGGCTCGAAGATTATTTTGCTAAATAGCATAAACTATT...


In [37]:
df_filt.shape[0]

1930

In [46]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
file_out='./diff_seg.fasta'

with open(file_out, 'w') as f_out:
    for i in range(df_filt.shape[0]):
        seq_record = SeqRecord(Seq(df_filt["seq"].iloc[i]))
        seq_record.id = df_filt["id"].iloc[i]
        seq_record.description = df_filt["id"].iloc[i]
        seq_record.name = df_filt["id"].iloc[i]
        # write new fasta file
        r=SeqIO.write(seq_record, f_out, 'fasta')
        if r!=1: 
            print('Error while writing sequence:  ' + seq_record.id)

SeqRecord(seq='TGGTATGAGATTGATGCTTTTGGACTACCAAATCAACTGCGGCAAAAGTTCGTTCAAAGTTCTATGGATGAAGAAACAGAGCAATTCTTAGCTTTGTGTGTCGAAAAGTCGGGATGGGTCTTTACTCAGTTCTTCCATTCTATTGTTCAAGTGTTCTTGGGTTTTATGGTTTCAATGACGACCATTAATGGGTTTCTTGAGCGAGGAAAGATGTTCGTATTTTCCAACGAGCAGTTCAAGCATTTATTGGAGATAGATGATTCATGCAGTAATAGAACACTGTTGGATCTTGGTGCTGGCGATGGTGCTGTAACACAACGCATGGCTGATCATTTCAAGAAAGTTTTCGTCACAGAAAGTTCATCAACTATGCGTATGAGGCTTGCACAAAAAGGATTTACGGTACTCAATATTGACGAATGGCAGAACTCGCACTATGACGTCATCAGTATGCTAAATCTTGTTGACAGAATAGAAAATCCATTACAGCTATTGAATGATGCACGATTATCATTAAATCGTGGTGGACTAGTGATCATTGCGATTGTTCTTCCATACAGTCCTGTAGTGGAAAAAGGTAGCAAATTCGTGCCACCAGAAGAAAAACTGCACATTTCTGGTAAAAGTGCCGAAGAGCAGATAAGAACATTTGTTATGGATGTATTAGAACCAGCTGGATTCGAAGTTCTGAAATTTACAAAACTTCCCTATCTTTGTGAAGGTGATTTATATGACGATTATTTTCTCCTCCATGATTACGTATTTGTGCTTAAATCAAGAATTGTGCTTGATTGATGATTATTAAATGATGATATCATGAAGAAATGTTCATGTTACTGTTGCAAGATGTTTGCAAGTCCGTAGTAAGATCTGTTACCTTCACTTTGGGACAACCGGTGCATGGTTGTAAGGCTTAGCCTCGCATTTTGTGGTCTGTCCATTGACTGTTGAAACTATTGAATTAACAGATTAACAGTGTTGAGTT